In [67]:
# import dependencies 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
# import tensorflow as tf

In [68]:
#  read in the csv
health_df = pd.read_csv('Resources/urgentcare_clean.csv')
health_df.head()

,diagnosis_category,diagnosis_sub_category,treatment_category,treatment_sub_category,determination,type,age_range,patient_gender
0,Infectious,Hepatitis,Pharmacy/Prescription Drugs,Anti-virals,Overturned Decision of Health Plan,Medical Necessity,41-50,Male
1,Mental,Eating Disorder,Mental Health Treatment,Residential Treatment Center - Admission,Upheld Decision of Health Plan,Medical Necessity,21-30,Female
2,Autism Spectrum,Autism-PDD-NOS,Autism Related Treatment,Speech Therapy,Upheld Decision of Health Plan,Medical Necessity,0-10,Female
3,Prevention/Good Health,NaN,"Diagnostic Imaging, Screening and Testing",Mammography,Overturned Decision of Health Plan,Experimental/Investigational,65+,Female
4,Prevention/Good Health,NaN,"Diagnostic Imaging, Screening and Testing",Lab Work,Upheld Decision of Health Plan,Experimental/Investigational,21-30,Male


In [69]:
# Determine data types and parameters
health_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1307 entries, 0 to 1306
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   diagnosis_category      1307 non-null   object
 1   diagnosis_sub_category  1171 non-null   object
 2   treatment_category      1306 non-null   object
 3   treatment_sub_category  1298 non-null   object
 4   determination           1307 non-null   object
 5   type                    1307 non-null   object
 6   age_range               1307 non-null   object
 7   patient_gender          1307 non-null   object
dtypes: object(8)
memory usage: 81.8+ KB


In [70]:
# Determine the number of unique values in each column.
data_cat = health_df.dtypes[health_df.dtypes == "object"].index.tolist()
health_df[data_cat].nunique()

# We'll need to bin the 'diagnosis_category, diagnosis_sub_category, treatment_category, and treatment_sub_category' 
# with 'other' categories 

diagnosis_category         27
diagnosis_sub_category    145
treatment_category         30
treatment_sub_category    120
determination               2
type                        3
age_range                   7
patient_gender              2
dtype: int64

In [71]:
# column 'determination' is the target data, meaning we will need to separate this from our data set and use 
# supervised learning to train our model. 
health_df['determination'].value_counts()

Overturned Decision of Health Plan    724
Upheld Decision of Health Plan        583
Name: determination, dtype: int64

# Fit all bins into 10 categories each 

# Fit "diagnosis_category" bin


In [72]:
# To clean all bins, we will first stary with the diagnosis_category
diagnoses = health_df['diagnosis_category'].value_counts()
diagnoses

Infectious                               244
Orthopedic/ Musculoskeletal              187
Mental                                   154
OB-Gyn/ Pregnancy                        108
Prevention/Good Health                   105
Cancer                                    86
Central Nervous System/ Neuromuscular     70
Cardiac/Circulatory                       57
Digestive System/ Gastrointestinal        48
Endocrine/ Metabolic                      36
Autism Spectrum                           31
Skin                                      28
Respiratory System                        23
Immunologic                               21
Genitourinary/ Kidney                     15
Pediatrics                                14
Chronic Pain                              12
Morbid Obesity                            11
Blood Related                              9
Foot                                       8
Ears, Nose, Throat                         8
Not Applicable                             8
Genetic   

In [73]:
# Choose a cutoff value and create a list of application types to be replaced (45)
# use the variable name `diagnosis_types_to_replace`

diagnosis_types_to_replace = diagnoses[diagnoses<45]
diagnosis_types_to_replace

Endocrine/ Metabolic          36
Autism Spectrum               31
Skin                          28
Respiratory System            23
Immunologic                   21
Genitourinary/ Kidney         15
Pediatrics                    14
Chronic Pain                  12
Morbid Obesity                11
Blood Related                  9
Foot                           8
Ears, Nose, Throat             8
Not Applicable                 8
Genetic                        7
Vision                         5
Trauma/Injuries                5
Dental                         5
Post Surgical Complication     2
Name: diagnosis_category, dtype: int64

In [74]:
# Checking the number of values that will go into 'other' column
diagnosis_types_to_replace = health_df['diagnosis_category'].isin(diagnosis_types_to_replace.index)
diagnosis_types_to_replace.sum()

248

In [75]:
# replace lower values of bin threshold with list comprehension 
health_df.loc[diagnosis_types_to_replace, 'diagnosis_category'] = 'Other'
health_df['diagnosis_category'].value_counts()

Other                                    248
Infectious                               244
Orthopedic/ Musculoskeletal              187
Mental                                   154
OB-Gyn/ Pregnancy                        108
Prevention/Good Health                   105
Cancer                                    86
Central Nervous System/ Neuromuscular     70
Cardiac/Circulatory                       57
Digestive System/ Gastrointestinal        48
Name: diagnosis_category, dtype: int64

# Fix 'diagnosis_sub_category' bin

In [76]:
health_df['diagnosis_sub_category'].value_counts()

Hepatitis                  233
Other                      196
Female Breast Disorder      47
Back Pain                   43
Breast Cancer               37
                          ... 
Carpal Tunnel Syndrome       1
Urinary Tract Infection      1
Decay/ Cavities              1
Viral Infection              1
Paralysis                    1
Name: diagnosis_sub_category, Length: 145, dtype: int64

In [77]:
# print a list of value counts to find a working threshold (20)
list(health_df['diagnosis_sub_category'].value_counts())

[233,
 196,
 47,
 43,
 37,
 29,
 23,
 22,
 22,
 21,
 19,
 19,
 17,
 16,
 16,
 15,
 15,
 13,
 13,
 12,
 12,
 11,
 11,
 11,
 9,
 9,
 9,
 8,
 8,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [78]:
# establish diagnosis_sub_cat as a list variable 
sub_diagnoses = health_df['diagnosis_sub_category'].value_counts()

# use a filter reduce list of bins 
sub_diagnosis_types_to_replace = sub_diagnoses[sub_diagnoses<20]
sub_diagnosis_types_to_replace

Fracture                      19
Substance Abuse/ Addiction    19
Thyroid Problems              17
Multiple Sclerosis            16
Crohn's Disease               16
                              ..
Carpal Tunnel Syndrome         1
Urinary Tract Infection        1
Decay/ Cavities                1
Viral Infection                1
Paralysis                      1
Name: diagnosis_sub_category, Length: 135, dtype: int64

In [79]:
# Checking the number of values that will go into 'other' column
sub_diagnosis_types_to_replace = health_df['diagnosis_sub_category'].isin(sub_diagnosis_types_to_replace.index)
sub_diagnosis_types_to_replace.sum()

498

In [80]:
# replace lower values of bin threshold with list comprehension 
health_df.loc[sub_diagnosis_types_to_replace, 'diagnosis_sub_category'] = 'Other'
health_df['diagnosis_sub_category'].value_counts()

Other                     694
Hepatitis                 233
Female Breast Disorder     47
Back Pain                  43
Breast Cancer              37
Autism-PDD-NOS             29
Depression - Severe        23
Eating Disorder            22
Knee Problem               22
Osteoarthritis             21
Name: diagnosis_sub_category, dtype: int64

# TO NOTE: 
First, this diagnosis sub categroy already has an 'Other' value, meaning that the values below our initial threshold were added an already existing category. Also, it's important to go back to the original DF, explore the data through visualizations (Tableau), and see if there are any immediate findings that we can tease out as it relates to emergency vs. frequency. 

# Next: 'treatment_category'

In [81]:
# Looking at this list, it seems important to perhaps come back and test the model without binning, 
# becuase there are list items that seem infrequent but important; or, at lest increase from 10 to 15-18 etc. 
health_df['treatment_category'].value_counts()

Pharmacy/Prescription Drugs                                       421
Diagnostic Imaging, Screening and Testing                         355
Mental Health Treatment                                           122
Durable Medical Equipment                                          53
Orthopedic                                                         41
Cancer Treatment                                                   30
Autism Related Treatment                                           29
Pain Management                                                    28
General Surgery                                                    28
Electrical/ Thermal/ Radiofreq. Interventions                      25
Reconstructive/Plastic Surgery                                     24
Special Procedure                                                  20
Diagnostic/Physician Evaluation                                    16
Rehabilitation Services - Skilled Nursing Facility - Inpatient     14
Emergency/Urgent Car

In [82]:
# establish treatment category as a list variable 
treatments = health_df['treatment_category'].value_counts()

# use a filter reduce list of bins 
treatment_types_to_replace = treatments[treatments<10]
treatment_types_to_replace

Home Health Care                   9
Ear, Nose and Throat Procedures    8
Dental/Orthodontic                 7
OB/GYN Procedures                  7
Urology                            5
Neurosugery                        4
Alternative Treatment              3
Not Applicable                     2
Vision                             2
Chiropractic                       2
Ophthalmology                      2
Preventive Health Screening        1
Name: treatment_category, dtype: int64

In [83]:
# Checking the number of values that will go into 'other' column
treatment_types_to_replace = health_df['treatment_category'].isin(treatment_types_to_replace.index)
treatment_types_to_replace.sum()

52

In [84]:
# replace lower values of bin threshold with list comprehension 
health_df.loc[treatment_types_to_replace, 'treatment_category'] = 'Other'
health_df['treatment_category'].value_counts()

Pharmacy/Prescription Drugs                                       421
Diagnostic Imaging, Screening and Testing                         355
Mental Health Treatment                                           122
Durable Medical Equipment                                          53
Other                                                              52
Orthopedic                                                         41
Cancer Treatment                                                   30
Autism Related Treatment                                           29
General Surgery                                                    28
Pain Management                                                    28
Electrical/ Thermal/ Radiofreq. Interventions                      25
Reconstructive/Plastic Surgery                                     24
Special Procedure                                                  20
Diagnostic/Physician Evaluation                                    16
Rehabilitation Servi

# Fit 'treatment_sub_category'

In [85]:
# print value counts to see threshold (19)
list(health_df['treatment_sub_category'].value_counts())
# health_df['treatment_sub_category'].value_counts()

[237,
 233,
 171,
 83,
 47,
 30,
 23,
 22,
 19,
 18,
 16,
 15,
 15,
 14,
 14,
 12,
 11,
 10,
 10,
 10,
 10,
 9,
 9,
 9,
 8,
 8,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 6,
 6,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [86]:
# establish treatment_sub_cat as a list variable 
sub_treatments = health_df['treatment_sub_category'].value_counts()

# use a filter reduce list of bins 
sub_treatment_types_to_replace = sub_treatments[sub_treatments<=18]
sub_treatment_types_to_replace

Partial Hospitalization                            18
X-Ray                                              16
Hormones                                           15
ABA-Applied Behavioral Analysis                    15
Skin Treatment                                     14
                                                   ..
Eye Test                                            1
Compression Garments                                1
EECP-Enhanced External Counter Pulsation Device     1
Psychotherapy                                       1
Medical/Surgical Unit                               1
Name: treatment_sub_category, Length: 111, dtype: int64

In [87]:
# Checking the number of values that will go into 'other' column
# This number is a little large; it may be worth 
sub_treatment_types_to_replace = health_df['treatment_sub_category'].isin(sub_treatment_types_to_replace.index)
sub_treatment_types_to_replace.sum()

433

In [88]:
# replace lower values of bin threshold with list comprehension 
health_df.loc[sub_treatment_types_to_replace, 'treatment_sub_category'] = 'Other'
health_df['treatment_sub_category'].value_counts()

Other                                       670
Anti-virals                                 233
Mammography                                 171
Lab Work                                     83
Residential Treatment Center - Admission     47
Acute Psychiatric Facility Admission         30
MRI                                          23
Analgesics                                   22
Arthritis Medications                        19
Name: treatment_sub_category, dtype: int64

In [17]:
health_df['type'].value_counts()

Medical Necessity               821
Experimental/Investigational    468
Urgent Care                      18
Name: type, dtype: int64

In [11]:
# Checking all other value counts to see if number of bins are appropriate for every columns 
bins = []

for column in health_df:
    

AttributeError: 'str' object has no attribute 'value_counts'